# LIGO Black Hole Echo Search Tutorial

## Testing the BULK Framework Prediction

This notebook walks through searching for gravitational wave echoes in LIGO data.

**Prediction:** Black hole mergers should show faint echoes 0.1-1.2 milliseconds after ringdown.

**Goal:** Test if this prediction holds using publicly available LIGO data.

---

### What You'll Learn:
1. How to load LIGO gravitational wave data
2. How to isolate the ringdown phase
3. How to search for delayed replicas (echoes)
4. How to assess statistical significance
5. How to interpret results

### Prerequisites:
- Basic Python knowledge
- Understanding of what gravitational waves are (helpful but not required)
- Patience and curiosity!

---

## Step 1: Import Libraries

First, we'll import the tools we need.

In [ ]:
# Standard scientific computing
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from scipy.stats import norm

# Gravitational wave tools
from gwpy.timeseries import TimeSeries

# Make plots look nice
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print("✓ Libraries imported successfully!")

## Step 2: Download LIGO Data

We'll use GW150914 - the first gravitational wave detection.

This was two black holes (36 and 29 solar masses) merging 1.3 billion light-years away.

In [ ]:
# Download data from Hanford detector
event_name = 'GW150914'
detector = 'H1'  # Hanford, Washington

print(f"Downloading {event_name} data from {detector}...")
data = TimeSeries.fetch_open_data(detector, event_name)

print(f"✓ Data loaded!")
print(f"  Duration: {data.duration:.1f} seconds")
print(f"  Sample rate: {data.sample_rate} Hz")
print(f"  Total samples: {len(data):,}")

## Step 3: Visualize the Raw Data

Let's see what the raw gravitational wave looks like.

In [ ]:
# Plot the full data stream
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(data.times, data.value, 'k-', linewidth=0.5)
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title(f'{event_name} - Raw Data from {detector}', fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Note: The signal is buried in noise - we'll need to filter it!")

## Step 4: Apply Bandpass Filter

Gravitational waves from black hole mergers are in the 30-400 Hz range.

We'll filter out everything else.

In [ ]:
# Apply bandpass filter
low_freq = 30   # Hz
high_freq = 400  # Hz

print(f"Filtering to {low_freq}-{high_freq} Hz...")
filtered_data = data.bandpass(low_freq, high_freq)
print("✓ Filtering complete!")

# Plot filtered data
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(filtered_data.times, filtered_data.value, 'b-', linewidth=0.8)
ax.set_xlabel('Time (seconds)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title(f'{event_name} - Filtered Data ({low_freq}-{high_freq} Hz)', fontsize=14)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Now we can see the signal much more clearly!")

## Step 5: Zoom In on the Merger

The actual gravitational wave event lasts less than a second.

Let's zoom in to see the three phases:
1. **Inspiral** - black holes spiraling inward
2. **Merger** - the collision
3. **Ringdown** - the combined black hole "ringing" like a bell

In [ ]:
# Find the peak (merger moment)
peak_idx = np.argmax(np.abs(filtered_data.value))
peak_time = filtered_data.times[peak_idx].value

print(f"Peak amplitude at GPS time: {peak_time:.3f}")

# Zoom in to ±0.2 seconds around peak
zoom_data = filtered_data.crop(peak_time - 0.2, peak_time + 0.2)

# Plot zoomed view
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot((zoom_data.times - peak_time)*1000, zoom_data.value, 'b-', linewidth=1.5)
ax.axvline(0, color='r', linestyle='--', label='Merger peak')
ax.set_xlabel('Time relative to merger (milliseconds)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title('Gravitational Wave Signal - Zoomed View', fontsize=14)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Annotate phases
ax.annotate('Inspiral', xy=(-150, 0.5e-21), fontsize=11, color='green')
ax.annotate('Merger', xy=(-10, 0.8e-21), fontsize=11, color='red')
ax.annotate('Ringdown', xy=(30, 0.4e-21), fontsize=11, color='blue')

plt.tight_layout()
plt.show()

## Step 6: Extract the Ringdown

The ringdown is where we'll search for echoes.

This is the "ringing" phase where the newly-formed black hole settles down.

In [ ]:
# Extract 100 ms of ringdown starting at peak
ringdown_duration = 0.1  # seconds
ringdown = filtered_data.crop(peak_time, peak_time + ringdown_duration)

print(f"Ringdown extracted:")
print(f"  Duration: {ringdown_duration*1000} ms")
print(f"  Samples: {len(ringdown)}")

# Plot ringdown template
fig, ax = plt.subplots(figsize=(12, 4))
times_ms = (ringdown.times - peak_time) * 1000  # Convert to milliseconds
ax.plot(times_ms, ringdown.value, 'k-', linewidth=1.5)
ax.set_xlabel('Time after merger (milliseconds)', fontsize=12)
ax.set_ylabel('Strain', fontsize=12)
ax.set_title('Ringdown Signal (Template for Echo Search)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nThis is the 'template' we'll search for as a delayed echo.")

## Step 7: Search for Echoes

Now the main event: searching for a delayed replica of the ringdown.

**Method:** Cross-correlation at different delay times.

**BULK prediction:** Echo should appear at 0.1-1.2 milliseconds after ringdown.

In [ ]:
# Search parameters (following BULK prediction)
delay_min = 0.0001   # 0.1 ms
delay_max = 0.0012   # 1.2 ms
delay_step = 0.00001  # 0.01 ms resolution

# Create array of delays to test
delays = np.arange(delay_min, delay_max, delay_step)
correlations = []

# Normalize template
template = ringdown.value
template_normalized = template / np.std(template)

print(f"Searching {len(delays)} delays from {delay_min*1000:.2f} to {delay_max*1000:.2f} ms...")

# Search each delay
sample_rate = filtered_data.sample_rate.value

for delay in delays:
    # Get data segment after ringdown (where echo should be)
    search_start = ringdown.times[-1].value + delay
    search_end = search_start + len(template) / sample_rate
    
    try:
        search_segment = filtered_data.crop(search_start, search_end).value
        search_normalized = search_segment / np.std(search_segment)
        
        # Cross-correlation
        correlation = np.corrcoef(template_normalized, search_normalized)[0, 1]
        correlations.append(correlation)
    except:
        correlations.append(0)

correlations = np.array(correlations)
print("✓ Search complete!")

## Step 8: Analyze Results

Find the peak correlation and assess its significance.

In [ ]:
# Find peak
peak_idx = np.argmax(np.abs(correlations))
peak_delay = delays[peak_idx]
peak_correlation = correlations[peak_idx]

# Calculate significance
noise_std = 1.0 / np.sqrt(len(template))  # Expected for random noise
significance = np.abs(peak_correlation) / noise_std

print("\n" + "="*60)
print("RESULTS:")
print("="*60)
print(f"Peak correlation: {peak_correlation:.6f}")
print(f"Peak delay: {peak_delay*1000:.3f} ms")
print(f"Expected noise level: ±{noise_std:.6f}")
print(f"Statistical significance: {significance:.2f} σ")
print("="*60)

if significance > 5:
    print("\n🎯 STRONG ECHO CANDIDATE (>5σ)")
    print("This warrants detailed investigation!")
elif significance > 3:
    print("\n⚠️  MARGINAL DETECTION (3-5σ)")
    print("Interesting, but needs more analysis.")
else:
    print("\nNo significant echo detected at this delay range.")
    print("Either no echo exists, or amplitude below threshold.")

## Step 9: Visualize Results

Create a detailed plot showing the correlation vs. delay.

In [ ]:
fig, ax = plt.subplots(figsize=(14, 6))

# Plot correlations
delays_ms = delays * 1000  # Convert to milliseconds
ax.plot(delays_ms, correlations, 'b-', linewidth=1.5, label='Cross-correlation')

# Mark significance thresholds
ax.axhline(noise_std, color='orange', linestyle='--', linewidth=1,
          label=f'1σ noise ({noise_std:.4f})')
ax.axhline(3*noise_std, color='red', linestyle='--', linewidth=1.5,
          label='3σ threshold')
ax.axhline(-noise_std, color='orange', linestyle='--', linewidth=1)
ax.axhline(-3*noise_std, color='red', linestyle='--', linewidth=1.5)

# Mark peak
ax.plot(peak_delay*1000, peak_correlation, 'go', markersize=12,
       label=f'Peak: {peak_delay*1000:.3f} ms ({significance:.2f}σ)')

# Shade BULK prediction range
ax.axvspan(0.1, 1.2, alpha=0.1, color='green', label='BULK predicted range')

ax.set_xlabel('Echo Delay (milliseconds)', fontsize=13)
ax.set_ylabel('Cross-Correlation', fontsize=13)
ax.set_title(f'{event_name} Echo Search Results - {detector} Detector',
            fontsize=15, fontweight='bold')
ax.legend(fontsize=10, loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Step 10: Interpretation

### What Does This Mean?

**If you see a peak > 3σ within 0.1-1.2 ms:**
- Potential echo detected!
- Need to verify with other detectors (L1, V1)
- Test on multiple events for consistency
- Rule out systematic errors
- This could be evidence for quantum structure at the horizon

**If no significant peak:**
- Either no echo exists at this timing
- Or echo amplitude is below detection threshold
- BULK port model might need revision
- Or impedance matching is nearly perfect

### Next Steps:

1. **Test other detectors:** Run same analysis on L1, V1
2. **Test other events:** GW170814, GW190521, etc.
3. **Vary parameters:** Try different delay ranges, filter settings
4. **Statistical rigor:** Bootstrap analysis, false alarm probability
5. **Document everything:** Save results, methodology, parameters

### How to Report Results:

**If you find something interesting:**
- Email: tsallie@gmail.com
- Open GitHub issue: https://github.com/tsallie52/TheBulkFramework
- Include: Event, detector, parameters, plots, significance

**Even null results are valuable!** They constrain theoretical models.

## Exercise: Try It Yourself

### Easy Modifications:

1. **Change the event:**
   ```python
   event_name = 'GW170814'  # Or GW190521, GW151226, etc.
   ```

2. **Change the detector:**
   ```python
   detector = 'L1'  # Try Livingston instead of Hanford
   ```

3. **Adjust search range:**
   ```python
   delay_min = 0.00005  # 0.05 ms
   delay_max = 0.002    # 2.0 ms
   ```

### Advanced Challenges:

1. **Compare multiple detectors:** Do H1 and L1 show consistent results?
2. **Calculate false alarm probability:** How often would noise create this peak?
3. **Bootstrap analysis:** Resample data, see if peak is stable
4. **Matched filtering:** Use more sophisticated signal processing
5. **Frequency analysis:** Does echo have same frequency content as ringdown?

### Research Questions:

1. Does echo timing scale with black hole mass?
2. Is echo stronger for higher mass ratios?
3. Does echo appear in all events or just some?
4. Can you rule out instrumental artifacts?

**The data is yours to explore!**

## Resources

### LIGO Data:
- GWOSC Portal: https://gwosc.org
- Event Catalog: https://gwosc.org/eventapi/
- Tutorials: https://gwosc.org/tutorials/

### Documentation:
- GWpy: https://gwpy.github.io
- PyCBC: https://pycbc.org
- NumPy/SciPy: https://scipy.org

### BULK Framework:
- GitHub: https://github.com/tsallie52/TheBulkFramework
- Predictions: See TESTABLE_PREDICTIONS.md
- Background: See AUTHOR_PERSPECTIVE.md

### Scientific Context:
- Echo searches: Abedi et al. arXiv:1612.00266
- BH ringdown: Berti et al., "Testing GR with GWs" reviews
- Quantum horizons: Cardoso & Pani, Nature Astronomy (2019)

---

## Final Thoughts

**You've just:**
- Downloaded real gravitational wave data
- Analyzed a black hole merger
- Searched for predicted echoes
- Applied rigorous statistical methods
- Tested a novel hypothesis

**This is real science.**

Whether you find echoes or not, you're contributing to our understanding of:
- Black hole physics
- Quantum gravity
- The nature of spacetime

**Keep exploring. Stay curious. Question everything.**

And remember: The goal is truth, not being right.

---

**Author:** Tom Sallie (BULK Framework)  
**Contact:** tsallie@gmail.com  
**License:** MIT (free to use, modify, share)

**Happy hunting! 🔭**